In [19]:
from google.colab import files
import io
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
#traning data imporation 
df=files.upload()
df=pd.read_csv(io.StringIO(df['train_data.csv'].decode('utf-8')))
df.head()

Saving train_data.csv to train_data.csv


,ID,parents,has_nurs,form,children,housing,finance,social,health,app_status
0,1,usual,less_proper,complete,3,critical,convenient,problematic,not_recom,0
1,2,pretentious,very_crit,completed,1,convenient,inconv,nonprob,not_recom,0
2,3,pretentious,proper,incomplete,1,less_conv,convenient,slightly_prob,priority,1
3,4,great_pret,improper,complete,1,convenient,convenient,nonprob,recommended,1
4,5,great_pret,less_proper,completed,1,convenient,convenient,slightly_prob,priority,1


In [ ]:
#traning data imporation 
dft=files.upload()
dft=pd.read_csv(io.StringIO(dft['test_data.csv'].decode('utf-8')))
dft.head()

Saving test_data.csv to test_data.csv


,ID,parents,has_nurs,form,children,housing,finance,social,health
0,1,usual,proper,complete,1,convenient,convenient,nonprob,recommended
1,2,usual,proper,complete,1,convenient,inconv,nonprob,priority
2,3,usual,proper,complete,1,convenient,inconv,problematic,priority
3,4,usual,proper,complete,1,less_conv,convenient,problematic,recommended
4,5,usual,proper,complete,1,less_conv,convenient,problematic,priority


In [ ]:
#Getting dummy varibles 
df1=pd.get_dummies(df)
df1=df1.set_index('ID')
df1t=pd.get_dummies(dft)
df1t=df1t.set_index('ID')

#Creating test and training data frames
X=df1.drop(['app_status'],1)
Y=df1['app_status']
Xt=df1t.copy()

#Apply into the logestic regression
logerg=LogisticRegression()
logerg.fit(X,Y)
Yt=logerg.predict(Xt)

#Check for the values
Yc=pd.DataFrame(Yt)
Yc.value_counts()

0    1663
1     929
dtype: int64

In [ ]:
 #Get the results in desired format 
 results=Yc
 results.index = np.arange(1, len(Yc) + 1)
 results=results.rename(columns={0:'app_status'})
 results.index=results.index.rename("ID")
 results.head()

,app_status
ID,
1,1
2,1
3,1
4,1
5,1


In [ ]:
#Download the results 
results.to_csv('LogisticRegression.csv')
files.download('LogisticRegression.csv')

### SVC

In [ ]:
#Getting dummy varibles 
df1=pd.get_dummies(df)
df1=df1.set_index('ID')
df1t=pd.get_dummies(dft)
df1t=df1t.set_index('ID')

#Denoting trainin varibles 
X=df1.drop(['app_status'],1)
X=np.array(X)
Y=df1['app_status']
Y=np.array(Y)

#creating SVC
rbf_svr=SVC(kernel='linear',C=0.1,gamma=1000)
rbf_svr.fit(X,Y)

#Denoting test varibles
Xt=np.array(df1t)
Yt=rbf_svr.predict(Xt)

#checking the data 
Yc=pd.DataFrame(Yt)
Yc.value_counts()

0    1684
1     908
dtype: int64

In [ ]:
 #Get the results in desired format 
 results=Yc
 results.index = np.arange(1, len(Yc) + 1)
 results=results.rename(columns={0:'app_status'})
 results.index=results.index.rename("ID")
 results.head()

,app_status
ID,
1,1
2,1
3,1
4,1
5,1


In [ ]:
results.to_csv('SVC_linear.csv')
files.download('SVC_linear.csv')

### Random forest

In [14]:
#2 parameters for the hyperparameter optimization 
n_estimators = [ int(x) for x in np.linspace(start=100, stop=1000, num=10)]
max_depth=[int(x) for x in np.linspace(start=10, stop =100, num=10)]
max_depth.append(None)

random_grid={'n_estimators':n_estimators, 'max_depth':max_depth}
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]}


In [23]:
#4 parameters for the hyperparamter optimization 
n_estimators = [ int(x) for x in np.linspace(start=100, stop=1000, num=10)]
max_depth=[int(x) for x in np.linspace(start=10, stop =100, num=10)]
max_depth.append(None)
min_samples_split = [2,6,10]
min_samples_leaf = [1,5,9]

random_grid={'n_estimators':n_estimators, 'max_depth':max_depth,  'min_samples_split':min_samples_split, 'min_samples_leaf':min_samples_leaf}
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 5, 9]}


In [15]:
#Getting dummy varibles 
df1=pd.get_dummies(df)
df1=df1.set_index('ID')
df1t=pd.get_dummies(dft)
df1t=df1t.set_index('ID')

#Creating training data frames
X=df1.drop(['app_status'],1)
Y=df1['app_status']

#Gridsearch hyperparamter optimization 
clf=RandomForestClassifier()
model=GridSearchCV(clf, random_grid,)
model.fit(X,Y)
model.best_params_


{'max_depth': 50, 'n_estimators': 200}

In [16]:
#assigning values 
clf=RandomForestClassifier(n_estimators=200, max_depth=50)
clf.fit(X,Y)

#Creating test data frames
Xt=df1t.copy()
Yt=clf.predict(Xt)

#Check the classification
Yc=pd.DataFrame(Yt)
Yc.value_counts()

0    1679
1     913
dtype: int64

In [17]:
#Get the result in required format 
results=Yc
results.index = np.arange(1, len(Yc) + 1)
results=results.rename(columns={0:'app_status'})
results.index=results.index.rename("ID")
results.head()

,app_status
ID,
1,1
2,1
3,1
4,1
5,1


In [18]:
#Download the CSV file 
results.to_csv('RF.csv')
files.download('RF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
#Confusion matrix for  data 
x_train, x_test, y_train, y_test =train_test_split(X,Y)
clf=RandomForestClassifier(n_estimators=200, max_depth=50)
clf.fit(x_train, y_train)

#Denoting the predicted and actual value 
YT=y_test
YP=clf.predict(x_test)

#confusion matrix
matrix = confusion_matrix(YT, YP, labels=[1,0])
print('Çonfusion matrix : \n ', matrix )

#F1 score
matrix = classification_report(YT,YP,labels=[1,0])
print('Classification report : \n',matrix)

Çonfusion matrix : 
  [[ 871   26]
 [   3 1692]]
Classification report : 
               precision    recall  f1-score   support

           1       1.00      0.97      0.98       897
           0       0.98      1.00      0.99      1695

    accuracy                           0.99      2592
   macro avg       0.99      0.98      0.99      2592
weighted avg       0.99      0.99      0.99      2592

